## Importing Packages

In [57]:
import pandas as pd
import os
import cv2

## --------------------------------------------------------------------------------------------------------------------------

## Exploring the CSV file

In [2]:
data = pd.read_csv("../data/external/size_estimation_homography_DeepFish.csv")

In [3]:
data.head()

,index,file,bbox,class,size (cm)
0,0,26_05_21-B20,"[2272.628996958517, 1685.2591150516498, 837.97...",Sepia officinalis,8.435085
1,1,26_05_21-B25,"[1807.393678506843, 1585.7146585117644, 1094.7...",Mullus barbatus,16.211420
2,2,26_05_21-B25,"[2648.726235342625, 1368.0723135423739, 345.68...",Mullus barbatus,14.422977
3,3,26_05_21-B25,"[2845.1096085383197, 1180.704037335252, 426.91...",Mullus barbatus,15.604945
4,4,26_05_21-B25,"[3192.0335090997633, 1235.4288323372514, 337.1...",Mullus barbatus,14.829357


#### Checking the number of images that contain fish of class "Pagrus pagrus" 

In [46]:
pagrus_images = []

for i in range(len(data)):
    if data.iloc[i]["class"] == "Pagrus pagrus":
        pagrus_images.append(data.iloc[i]["file"])
pagrus_images = set(pagrus_images)
print(f"There are {len(pagrus_images)} images of class Pagrus pagrus")

There are 166 images of class Pagrus pagrus


#### Checking whether all pagrus images in the deepfish dataset exists in the `data/external/all-pagrus-images` directory or not

In [52]:
all_images = []
missing_images = []
for image_file in os.listdir("../data/external/all-pagrus-images"):
    image_name = os.path.splitext(image_file)[0]
    all_images.append(image_name)
cond = True
for image in pagrus_images:
    if image not in all_images:
        cond = False
        missing_images.append(image)

if cond:
    print("All images in the deepfish dataset of the class Pagrus are in data/external/all-pagrus-images")
else:
    print(f"{len(missing_images)} images in the deepfish dataset of the class Pagrus are not in data/external/all-pagrus-images\n")
    print("Missing Images:\n", missing_images)

17 images in the deepfish dataset of the class Pagrus are not in data/external/all-pagrus-images

Missing Images:
 ['19_04_21-B11', '22_04_21-B53', '20_05_21-B9', '5_05_21-B45', '22_04_21-B26', '20_05_21-B27', '12_04_21-B.3', '24_05_21-B1', '17_05_21-B9', '19_04_21-B27', '23_04_21-B1', '20_05_21-B22', '11_06_21-B1', '18_05_21-B27', '31_05_21-B6', '3_05_21-B25', '13_04_21-B.6']


#### Moving the missing images to the `data/external/all-pagrus-images` directory

In [58]:
for image_file in os.listdir("../data/external/all-fish-images"):
    image_name = os.path.splitext(image_file)[0]
    if image_name in missing_images:
        image = cv2.imread("../data/external/all-fish-images/"+image_file)
        cv2.imwrite("../data/external/all-pagrus-images/"+image_file, image)

#### Checking again whether all pagrus images in the deepfish dataset exists in the `data/external/all-pagrus-images` directory or not

In [59]:
all_images = []
missing_images = []
for image_file in os.listdir("../data/external/all-pagrus-images"):
    image_name = os.path.splitext(image_file)[0]
    all_images.append(image_name)
cond = True
for image in pagrus_images:
    if image not in all_images:
        cond = False
        missing_images.append(image)

if cond:
    print("All images in the deepfish dataset of the class Pagrus are in data/external/all-pagrus-images")
else:
    print(f"{len(missing_images)} images in the deepfish dataset of the class Pagrus are not in data/external/all-pagrus-images\n")
    print("Missing Images:\n", missing_images)

All images in the deepfish dataset of the class Pagrus are in data/external/all-pagrus-images


In [62]:
import cv2
import numpy as np
from PIL import Image, ImageEnhance, ImageFilter

def add_blue_tint(image):
    blue_tint = image.copy()
    blue_tint[:, :, 0] = cv2.addWeighted(image[:, :, 0], 0.5, np.zeros(image[:, :, 0].shape, image.dtype), 0.5, 50)
    return blue_tint

def add_wave_distortion(image):
    rows, cols = image.shape[:2]
    src_points = np.float32([[0, 0], [cols - 1, 0], [0, rows - 1], [cols - 1, rows - 1]])
    dst_points = src_points + np.random.uniform(-10, 10, src_points.shape).astype(np.float32)
    warp_matrix = cv2.getPerspectiveTransform(src_points, dst_points)
    return cv2.warpPerspective(image, warp_matrix, (cols, rows))

def add_blur(image):
    return cv2.GaussianBlur(image, (15, 15), 0)

# Load image pagrus-images-with-water-effect
for image_file in os.listdir("../data/external/all-pagrus-images"):
    image = cv2.imread("../data/external/all-pagrus-images/"+image_file)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Apply blue tint
    image = add_blue_tint(image)

    # Apply wave distortion
    image = add_wave_distortion(image)

    # Apply blur
    image = add_blur(image)

    # Convert back to BGR for OpenCV
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Save the result
    cv2.imwrite("../data/external/pagrus-images-with-water-effect/"+image_file, image)
